In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

# 30000 line start

In [2]:
t0 = time.time()

In [3]:
# FOR TRAINING FROM RUSSIAN TO POLISH
# russian_data=pd.read_csv("data/ru/ru.txt",sep='\n',header=None)
# polish_data=pd.read_csv("data/pl/pl.txt",sep='\n',header=None)
# df = pd.DataFrame(columns=['ru','pl'])

# FOR TRAINING FROM POLISH TO RUSSIAN
russian_data=pd.read_csv("data/pl/pl.txt",sep='\n',header=None)
polish_data=pd.read_csv("data/ru/ru.txt",sep='\n',header=None)
df = pd.DataFrame(columns=['pl','ru'])

In [4]:
for a in range(len(polish_data)):
        df.loc[a] = russian_data[0][a].split(" "),polish_data[0][a].split(" ")

del polish_data
del russian_data

pl_ru_pair = df.to_numpy()
del df

In [5]:
# Extract dutch and english vocabularies
def vocab(eng_ru_pair):
    russian_words = []
    polish_words = []

    for s_p in pl_ru_pair:
        for ew in s_p[1]: 
            polish_words.append(ew)
        for fw in s_p[0]: 
            russian_words.append(fw)

    polish_words = sorted(list(set(polish_words)), key=lambda s: s.lower()) 
    russian_words = sorted(list(set(russian_words)), key=lambda s: s.lower())
    # print('Polish vocab: ', polish_words)
    # print('Russian vocab: ', russian_words)
    return polish_words,russian_words



In [6]:
# Vocabulary size of the english and dutch words
def vocab_size(polish_words,russian_words):
    polish_vocab_size = len(polish_words)
    russian_vocab_size = len(russian_words)
    print('polish_vocab_size: ', polish_vocab_size)
    print('russian_vocab_size: ', russian_vocab_size)
    return polish_vocab_size,russian_vocab_size



In [7]:
# Routine to uniformly initialize word translation probabilities in t hash
def init_prob(t, prev, init_val, pl_ru_pair):
    for a in range(len(pl_ru_pair)):
        for r_word in pl_ru_pair[a][0]:
            for e_word in pl_ru_pair[a][1]:
                t[(e_word, r_word)] = init_val
                prev[(e_word, r_word)] = 0


# Main routine
polish_words,russian_words = vocab(pl_ru_pair)
polish_vocab_size,russian_vocab_size = vocab_size(polish_words,russian_words)



polish_vocab_size:  44658
russian_vocab_size:  42745


In [8]:
# Initialize probabilities uniformly
final_total = {}
t = {}
prev = {}
init_val = 1.0 / russian_vocab_size
init_prob(t, prev, init_val, pl_ru_pair)

print('No. of russian/polish pairs: ', len(t))


delta = 1 
epsilon = 0.02

number_of_epochs = 0
# Loop while not converged
while (delta > epsilon):

    # Initialize
    count = {}
    total = {}
    pair_delta = 0
    
    for sp in pl_ru_pair:
        for fw in sp[0]:
            total[fw] = 0.0
            for ew in sp[1]:
                count[(ew,fw)] = 0.0
        
    for sp in pl_ru_pair:

        # Compute normalization
        for ew in sp[1]:
            final_total[ew] = 0.0
            for fw in sp[0]:
                final_total[ew] += t[(ew, fw)]

        # Collect counts
        for ew in sp[1]:
            for fw in sp[0]:
                count[(ew, fw)] += t[(ew, fw)] / final_total[ew]
                total[fw] += t[(ew, fw)] / final_total[ew]

                
    for a in range(len(pl_ru_pair)):
        for e_word in pl_ru_pair[a][1]:
            for r_word in pl_ru_pair[a][0]:
                pair_delta = max(pair_delta, abs(prev[(e_word, r_word)]-t[(e_word, r_word)]))
                prev[(e_word, r_word)] = t[(e_word, r_word)]
                t[(e_word, r_word)] = count[(e_word, r_word)] / total[r_word]
                
    
    number_of_epochs +=1
    delta = pair_delta
    print(delta)
    if(number_of_epochs>50):
        break

print("number_of_epochs",number_of_epochs)

#Finding the maximum matrix
print("No. of russian/polish pairs:",len(t))

convert = {}
coverted_prob = {}

for p_ed in t.keys():
    if p_ed[1] not in coverted_prob.keys():
        coverted_prob[p_ed[1]] = 0
    if(coverted_prob[p_ed[1]] < t[p_ed]):
        coverted_prob[p_ed[1]] = t[p_ed]
        convert[p_ed[1]] = p_ed[0]

np.save('pl_to_ru_20k.npy',convert)

t1 = time.time()
print("Time taken to train the model",time.strftime('%H:%M:%S', time.gmtime(t1-t0)))

No. of russian/polish pairs:  5214497
0.9999766054509299
0.9999766054509299
0.4064055753304225
0.23125982570232062
0.15868122901803722
0.12121628583901017
0.11177520065951979
0.08614411332725525
0.07692220376620923
0.061989161358031675
0.05475398356657965
0.04685599086376124
0.03872216104896753
0.03339846924834716
0.038777908447023335
0.03583602943918601
0.0377968787324796
0.041576715832799954
0.03992868640570271
0.03467322273967452
0.028424743211989928
0.022800575851583793
0.01830600484398065
number_of_epochs 23
No. of russian/polish pairs: 5214497
Time taken to train the model 00:34:54
